# Big Data in Finance
## Understanding Mutual Fund Data
### Instructor :
Simona Abis, simona.abis@gsb.columbia.edu

# Module Objectives
## Lecture 1:
1. Download data and understand data structure
2. Clean the main data file and sub-set it to identify balanced, US active equity funds 
    - **(ID: {sub-fund, quarter})**
3. Aggregate the information in the identified sub-set by share classes 
    - **(ID: FROM {sub-fund, quarter} TO {fund, quarter})**

## Homework 1:
1. Think through ID mapping to ensure greatest possible data coverage 
    - crsp_fundno = sub-fund (share classes)
    - crsp_cl_grp = fund (unit of interest) 
    - crsp_portno = portfolio identifier (CRSP) 
    - wficn = portfolio identifier (Thomson Reuters)
2. Create and forward fill a map derived from the obtained aggregate file to be used in sub-setting monthly returns & holdings information 
    - **(ID: {fund, month})**

## Lecture 2: 
Utilizing the map developed in the homework
1. Sub-set, clean and aggregate monthly returns
    - **(ID: FROM {sub-fund, month} TO {fund, month})**
2. Clean, sub-set and complement holdings data to retrieve comprehensive holdings information 
    - **(ID: FROM {portfolio, date} TO {fund date})**


## Homework 2:
1. Generate holdings & returns based performance measures
    - **(ID: fund, month)**
2. Aggregate all information into a monthly panel dataset 
    - **(ID: fund, month)**

# Lecture 1.1
# Download & Understand the data

## Database information:
- Download CRSP Mutual fund guide at: http://www.crsp.com/files/MFDB_Guide.pdf 
- Check online data guide at: http://www.crsp.com/products/research-products/crsp-survivor-bias-free-us-mutual-funds

## Download datasets from CRSP via WRDS Python API
- WRDS: Querying with Python (Passing SQL Params) : https://wrds-www.wharton.upenn.edu/pages/support/programming-wrds/programming-python/querying-wrds-data-python/
- All table/variable names are available on: https://wrds-web.wharton.upenn.edu/wrds/tools/variable.cfm?vendor_id=10

## Import Settings

### Adjust paths in 'settings.py' to reflect local machine
- Open the settings.py file found in the head directory
- Change the 'user' and 'basepath' variables to reflect where you would like to store files in your local machine
- The code will create 'Input' and output ('dbcourse_output') folders in the chosen directory
    - Raw data from CRSP will be saved in the input folder through the WRDS python API
    - Intermediate outputs will be saved in the output folder

### Load settings and other needed packages

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)
from Functions import Utilis as Util
from settings import *

# Import & Save datasets

## Initialize database connection

In [ ]:
# Make sure you set up your wrds connection first (see head directory: wrds_setup.py)
db = wrds.Connection(wrds_username='sa3518')

## Datasets needed for Lecture 1

### get 'fund_summary_full.csv', (03/2008 - 06/2010)

In [ ]:
exists = os.path.isfile(inputPath + 'fund_summary_full.csv')
if exists:
    t0 = time.time()
    print('File already exists locally, loading file...')
    df = pd.read_csv(inputPath + fund_summary_name, low_memory=False)
    t1 = time.time()
    print('File loaded successfully in', t1-t0, 's')
else:
    print('Downloading fund_summary_full...')
    t0 = time.time()
    df_fund_summary_query = "SELECT * FROM crspq.fund_summary2 WHERE (caldt BETWEEN '2008-03-01' AND '2010-07-01')"
    df = db.raw_sql(df_fund_summary_query)
    df.to_csv(inputPath + "fund_summary_full.csv", index=False)
    t1 = time.time()
    print("Downloaded fund_summary_full Successfully in", t1-t0, 's')

#### fund_summary_full: Dataset Description
- Key variables: CRSP_FUNDNO, CALDT
    - CRSP_FUNDNO: Identifies sub-funds at the share class level
    - CALDT: Calendar date at quarter end
- Contains the main descriptive info about funds as well as TNA, expense ratio and turnover rate

In [ ]:
print('The shape of the dataset is:', df.shape)
print('The number of unique crsp_fundno is:', df.crsp_fundno.nunique())
print('The number of unique crsp_cl_grp is:', df.crsp_cl_grp.nunique())
print('The average number of crsp_fundno per crsp_cl_grp per date:', 
      df.groupby(['crsp_cl_grp', 'caldt'])['crsp_fundno'].nunique().mean())
print('The average number of crsp_fundno per crsp_cl_grp over time:', 
      df.groupby(['crsp_cl_grp'])['crsp_fundno'].nunique().mean())
df.head(4)

### get 'FrontLoads.csv', (year: full available)

In [ ]:
exists = os.path.isfile(inputPath + 'FrontLoads.csv')
if exists:
    t0 = time.time()
    print('File already exists locally, loading file...')
    df = pd.read_csv(inputPath + FRONT_LOAD_name, low_memory=False)
    t1 = time.time()
    print('File loaded successfully in', t1-t0, 's')
else:
    print('Downloading FrontLoads...')
    t0 = time.time()
    df = db.get_table(library='crspq', table='front_load')
    df.to_csv(inputPath + "FrontLoads.csv", index=False)
    t1 = time.time()
    print('Downloaded FrontLoads Successfully in', t1-t0, 's')

#### FrontLoads: Dataset Description
- Key variables: CRSP_FUNDNO, begdt, enddt
    - CRSP_FUNDNO: Identifies sub-funds at the share class level
    - begdt: beginning date for which observations are relevant
    - enddt: end date for which observations are relevant
- Contains information about front loads (a type of fee)

In [ ]:
print('The shape of the dataset is:', df.shape)
df.head(4)

### get 'RearLoads.csv', (year: full available)

In [ ]:
exists = os.path.isfile(inputPath + 'RearLoads.csv')
if exists:
    t0 = time.time()
    print('File already exists locally, loading file...')
    df = pd.read_csv(inputPath + REAR_LOAD_name, low_memory=False)
    t1 = time.time()
    print('File loaded successfully in', t1-t0, 's')
else:
    print('Downloading RearLoads')
    t0 = time.time()
    df = db.get_table(library='crspq', table='rear_load')
    df.to_csv(inputPath + "RearLoads.csv", index=False)
    t1 = time.time()
    print('Downloaded Rear Loads Successfully in', t1-t0, 's')

#### RearLoads: Dataset Description
- Key variables: CRSP_FUNDNO, begdt, enddt
- Contains read loads (a type of fee). 
    - These will have to be integrated with front loads to calculate total loads

In [ ]:
print('The shape of the dataset is:', df.shape)
df.head(4)

### get 'Map_fundno_portno.csv', (year: full available)

In [ ]:
exists = os.path.isfile(inputPath + 'Map_fundno_portno.csv')
if exists:
    t0 = time.time()
    print('File already exists locally, loading file...')
    df = pd.read_csv(inputPath + map_fundno_portno_name, low_memory=False)
    t1 = time.time()
    print('File loaded successfully in', t1-t0, 's')
else:
    print('Downloading Map_fundno_portno')
    t0 = time.time()
    df = db.get_table(library='crspq', table='portnomap')
    df.to_csv(inputPath + "Map_fundno_portno.csv", index=False)
    t1 = time.time()
    print("Downloaded Map_fundno_portno Successfully in", t1-t0, 's')

#### Map_fundno_portno: Dataset Description
- **Key variables**: *CRSP_FUNDNO, begdt, enddt*
- Maps *CRSP_FUNDNO* to *CRSP_PORTNO* (the CRSP portfolio identifier)
    - The mapping is **not unique** either by fund or over time
    - *begdt*: beginning date from which the mapping is vallid
    - *enddt*: end date until which the mapping is valid
- Needed to map funds to CRSP holdings

In [ ]:
print('The shape of the dataset is:', df.shape)
print('The average number of crsp_portno per crsp_fundno (over time):', 
      df.groupby('crsp_fundno')['crsp_portno'].nunique().mean())
print('The average number of crsp_fundno with the same crsp_portno is:', 
      df.groupby('crsp_portno')['crsp_fundno'].nunique().mean())
df.head(2)

### get 'Mflinks_crsp.csv', (year: full available)

In [ ]:
exists = os.path.isfile(inputPath + 'Mflinks_crsp.csv')
if exists:
    t0 = time.time()
    print('File already exists locally, loading file...')
    df = pd.read_csv(inputPath + CRSP_FUNDNO_WFICN_name, low_memory=False)
    t1 = time.time()
    print('File loaded successfully in', t1-t0, 's')
else:
    print('Downloading Mflinks_crsp')
    t0 = time.time()
    df = db.get_table(library='mfl', table='mflink1')
    df.to_csv(inputPath + "Mflinks_crsp.csv", index=False)
    t1 = time.time()
    print("Downloaded Mflinks_crsp Successfully in", t1-t0, 's')

#### Mflinks_crsp: Dataset Description
- **Key variables**: *CRSP_FUNDNO, WFICN*
- Maps *CRSP_FUNDNO* to *WFICN* (the Thomson Reuters portfolio identifier)
    - The mapping is **not** unique by fund but it **is** unique over time
- Needed to map funds to Thomson holdings

In [ ]:
print('The shape of the dataset is:', df.shape)
print('The average number of wficn per crsp_fundno (over time):', df.groupby('crsp_fundno')['wficn'].nunique().mean())
print('The average number of crsp_fundno with the same wficn is:', df.groupby('wficn')['crsp_fundno'].nunique().mean())
df.head(2)

## Datasets needed for Lecture 2:

### get 'MonthlyReturns.csv', (year: 03/2008-06/2010)

In [ ]:
exists = os.path.isfile(inputPath + 'MonthlyReturns.csv')
if exists:
    t0 = time.time()
    print('File already exists locally, loading file...')
    df = pd.read_csv(inputPath + monthly_return_name, low_memory=False)
    t1 = time.time()
    print('File loaded successfully in', t1-t0, 's')
else:
    print('Downloading MonthlyReturns')
    t0 = time.time()
    df_query = "SELECT * FROM crspq.monthly_tna_ret_nav WHERE (caldt BETWEEN '2008-03-01' AND '2010-07-01')"
    df = db.raw_sql(df_query)
    df.to_csv(inputPath + "MonthlyReturns.csv", index=False)
    t1 = time.time()
    print("Downloaded MonthlyReturns Successfully in", t1-t0, 's')

#### MonthlyReturns: Dataset Description
- **Key variables**: *crsp_fundno, caldt*
    - CRSP_FUNDNO: Identifies sub-funds at the share class level
    - CALDT: calendar date monthly
- Contains returns and TNA information at the monthly level

In [ ]:
print('The shape of the dataset is:', df.shape)
print('The number of unique crsp_fundno is:', df['crsp_fundno'].nunique())
df.head(2)

### get "StockHeaderInfo.csv", (year: full available)

In [ ]:
exists = os.path.isfile(inputPath + 'StockHeaderInfo.csv')
if exists:
    t0 = time.time()
    print('File already exists locally, loading file...')
    df = pd.read_csv(inputPath + "StockHeaderInfo.csv", low_memory=False)
    t1 = time.time()
    print('File loaded successfully in', t1-t0, 's')
else:
    print('Downloading StockHeaderInfo')
    t0 = time.time()
    df = db.get_table(library='crspq', table='dsfhdr')
    df.to_csv(inputPath + "StockHeaderInfo.csv", index=False)
    t1 = time.time()
    print("Downloaded StockHeaderInfo Successfully in", t1-t0, 's')

#### StockHeaderInfo: Dataset Description
- **Key variables**: *PERMNO, various begin and end dates*
    - *PERMNO*: CRSP's unique and permanent issue identification number
- Contains descriptive information about CRSP stocks with the time period for which they are relevant

In [ ]:
print('The shape of the dataset is:', df.shape)
df.head(2)

### get 'MonthlyStockFile.csv', (year: 03/2008-06/2010)

In [ ]:
exists = os.path.isfile(inputPath + 'MonthlyStockFile.csv')
if exists:
    t0 = time.time()
    print('File already exists locally, loading file...')
    df = pd.read_csv(inputPath + stocks_monthly_name, low_memory=False)
    t1 = time.time()
    print('File loaded successfully in', t1-t0, 's')
else:
    print('Downloading MonthlyStockFile')
    t0 = time.time()
    df_query = "SELECT * FROM crspq.msf WHERE (crspq.msf.date BETWEEN '2008-03-01' AND '2010-07-01')"
    df = db.raw_sql(df_query)
    df.to_csv(inputPath + "MonthlyStockFile.csv", index=False)
    t1 = time.time()
    print("Downloaded MonthlyStockFile Successfully in", t1-t0, 's')

#### MonthlyStockFile: Dataset Description
- **Key variables**: *permno, date*
    - *permno* : permanent stock identifier
    - *date* : date, monthly
- Contains price, return, ... information about US stocks


In [ ]:
print('The shape of the dataset is:', df.shape)
print('The number of unique stocks is:', df['permno'].nunique())
df.head(2)

### get "Mflinks_thompson.csv", (year: full available)

In [ ]:
exists = os.path.isfile(inputPath + 'Mflinks_thompson.csv')
if exists:
    t0 = time.time()
    print('File already exists locally, loading file...')
    df = pd.read_csv(inputPath + FUNDNO_WFICN_name, low_memory=False)
    t1 = time.time()
    print('File loaded successfully in', t1-t0, 's')
else:
    print('Downloading Mflinks_thompson')
    t0 = time.time()
    df = db.get_table(library='mfl', table='mflink2')
    df.to_csv(inputPath + "Mflinks_thompson.csv", index=False)
    t1 = time.time()
    print("Downloaded Mflinks_thompson Successfully in", t1-t0, 's')

#### Mflinks_thompson: Dataset Description
- **Key variables**: *FUNDNO, WFICN, fdate*
- *FUNDNO*: is the fund level identifier in Thomson Reuters holdings data
- Maps *FUNDNO* to *WFICN*
    - The mapping is **not unique** by fund or over time
- Needed to map funds information to Thomson holdings

In [ ]:
print('The shape of the dataset is:', df.shape)
print('The average number of wficn per fundno (over time):', df.groupby('fundno')['wficn'].nunique().mean())
print('The average number of fundno with the same wficn is:', df.groupby('wficn')['fundno'].nunique().mean())
Util.isnull_chk(df, 'wficn')
Util.isnull_chk(df, 'fundno')
df.head(2)

### get 'thompsonmasterfile.csv', (year: 12/2007-03/2008)

In [ ]:
exists = os.path.isfile(inputPath + 'thompsonmasterfile.csv')
if exists:
    t0 = time.time()
    print('File already exists locally, loading file...')
    df = pd.read_csv(inputPath + thomson_Holdings_name, low_memory=False)
    t1 = time.time()
    print('File loaded successfully in', t1-t0, 's')
else:
    print('Downloading thompsonmasterfile')
    t0 = time.time()
    df_query = "SELECT * FROM tfn.s12 WHERE (rdate BETWEEN '2007-12-01' AND '2008-04-01')"
    df = db.raw_sql(df_query)
    df.to_csv(inputPath + "thompsonmasterfile.csv", index=False)
    t1 = time.time()
    print("Downloaded thompsonmasterfile Successfully in", t1-t0, 's')

#### thompsonmasterfile: Dataset Description
- **Key variables**: *fundno, fdate, cusip*
    - *fundno* : Thomson fund identifier
    - *fdate* : date of filing
    - *cusip* : stock identifier
- Contains holdings information from Thomson Holdings

In [ ]:
print('The shape of the dataset is:', df.shape)
print('The number of unique funds is:', df[['fundno', 'fdate']].nunique())
df.head(2)

### get 'CRSPHoldings.csv', (year: 03/2008-06/2010)

In [ ]:
exists = os.path.isfile(inputPath + CRSP_Holdings_name)
if exists:
    t0 = time.time()
    print('File already exists locally, loading file...')
    df = pd.read_csv(inputPath + CRSP_Holdings_name, low_memory=False)
    t1 = time.time()
    print('File loaded successfully in', t1-t0, 's')
else:
    print('Downloading CRSP_Holdings...')
    t0 = time.time()
    df_query_holdings = "SELECT * FROM crspq.holdings WHERE (eff_dt BETWEEN '2008-03-01' AND '2010-07-01')"
    df = db.raw_sql(df_query_holdings)
    df.to_csv(inputPath + "CRSPHoldings" + ".csv", index=False)
    t1 = time.time()
    print("Downloaded CRSP Holding Successfully in", t1-t0, 's')


In [ ]:
"""
Download crsp_holdings year by year (if you don't want to wait for a long time

df_query_list = [
    "SELECT * FROM crspq.holdings WHERE (eff_dt BETWEEN '2008-03-01' AND '2008-12-31')",
    "SELECT * FROM crspq.holdings WHERE (eff_dt BETWEEN '2009-01-01' AND '2009-12-31')",
    "SELECT * FROM crspq.holdings WHERE (eff_dt BETWEEN '2010-01-01' AND '2010-07-01')"]

for i in range(len(df_query_list)):
    exists = os.path.isfile(inputPath + 'CRSP_Holdings'+ str(2008+i) + '.csv')
    print ('year' + str(2008 + i))
    if exists:
        print('File exists')
    else:
        t0 = time.time()
        print('Downloading CRSP_Holdings' + str(2008 + i) + '...')
        df = db.raw_sql(df_query_list[i])
        df.to_csv("CRSPHoldings" + str(2008 + i) + ".csv", index=False)
        t1 = time.time()
        print("Downloaded CRSP Holding Successfully in", t1-t0, 's')
"""

#### CRSPHoldings: Dataset Description
- **Key variables**: *crsp_portno, report_dt, cusip*
    - *crsp_portno* : portfolio number is the unique identifier for a security or a group of securities held in a fund’s portfolio
    - *report_dt* : date to which the holding correspond to
    - *cusip* : stock identifier

In [ ]:
print('The shape of the dataset is:', df.shape)
print('The number of unique portnos is:', df[['crsp_portno']].nunique())
print('The list of unique dates is:')
print(df.report_dt.unique())

In [ ]:
df.head(20)

# Close Database Connection

In [ ]:
db.close()
print('WRDS connection closed!')